In [1]:
import time
import argparse
import os
import torch
import numpy as np
import cv2
from skimage.measure import compare_ssim, compare_psnr

import utils
import dataset

def str2bool(v):
    #print(v)
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Unsupported value encountered.')

if __name__ == "__main__":
    # ----------------------------------------
    #        Initialize the parameters
    # ----------------------------------------
    parser = argparse.ArgumentParser()
    #GPU parameters
    parser.add_argument('--no_gpu', default = False, help = 'True for CPU')
    # Saving, and loading parameters
    #parser.add_argument('--save_name', type = str, default = './results/models_SA_onlySA2conv', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SA/KPN_rainy_image_epoch250_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SA_maxpooing/KPN_rainy_image_epoch250_bs8.pth', help = 'load the pre-trained model with certain epoch')
    
    #原KPN
    parser.add_argument('--save_name', type = str, default = './results/models_k3_d4_ssimloss', help = 'save the generated with certain epoch')
    parser.add_argument('--load_name', type = str, default = './models/models_k3_d4_ssimloss/KPN_rainy_image_epoch250_bs16.pth', help = 'load the pre-trained model with certain epoch')
    
    #models_SA_onlySA2conv
    #parser.add_argument('--save_name', type = str, default = './results/models_SA_onlySA2conv', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SA_onlySA2conv/KPN_rainy_image_epoch250_bs16.pth', help = 'load the pre-trained model with certain epoch')
    
    #parser.add_argument('--load_name', type = str, default = './models/KPN_single_image_epoch120_bs16_mu0_sigma30.pth', help = 'load the pre-trained model with certain epoch')
    parser.add_argument('--test_batch_size', type = int, default = 1, help = 'size of the batches')
    parser.add_argument('--num_workers', type = int, default = 1, help = 'number of workers')
    # Initialization parameters
    parser.add_argument('--color', type = str2bool, default = True, help = 'input type')
    parser.add_argument('--burst_length', type = int, default = 1, help = 'number of photos used in burst setting')
    parser.add_argument('--blind_est', type = str2bool, default = True, help = 'variance map')
    parser.add_argument('--kernel_size', type = list, default = [3], help = 'kernel size')
    parser.add_argument('--sep_conv', type = str2bool, default = False, help = 'simple output type')
    parser.add_argument('--channel_att', type = str2bool, default = False, help = 'channel wise attention')
    parser.add_argument('--spatial_att', type = str2bool, default = False, help = 'spatial wise attention')
    parser.add_argument('--upMode', type = str, default = 'bilinear', help = 'upMode')
    parser.add_argument('--core_bias', type = str2bool, default = False, help = 'core_bias')
    parser.add_argument('--init_type', type = str, default = 'xavier', help = 'initialization type of generator')
    parser.add_argument('--init_gain', type = float, default = 0.02, help = 'initialization gain of generator')
    # Dataset parameters
    parser.add_argument('--baseroot', type = str, default = './rainy_image_dataset/rain100H/test/', help = 'images baseroot')
    parser.add_argument('--crop', type = str2bool, default = False, help = 'whether to crop input images')
    parser.add_argument('--crop_size', type = int, default = 512, help = 'single patch size')
    parser.add_argument('--geometry_aug', type = str2bool, default = False, help = 'geometry augmentation (scaling)')
    parser.add_argument('--angle_aug', type = str2bool, default = False, help = 'geometry augmentation (rotation, flipping)')
    parser.add_argument('--scale_min', type = float, default = 1, help = 'min scaling factor')
    parser.add_argument('--scale_max', type = float, default = 1, help = 'max scaling factor')
    parser.add_argument('--add_noise', type = str2bool, default = False, help = 'whether to add noise to input images')
    parser.add_argument('--mu', type = int, default = 0, help = 'Gaussian noise mean')
    parser.add_argument('--sigma', type = int, default = 30, help = 'Gaussian noise variance: 30 | 50 | 70')
    opt = parser.parse_args(args=[])
    print(opt)
    
#   os.environ["CUDA_VISIBLE_DEVICES"] = "5"
    os.environ["CUDA_VISIBLE_DEVICES"] = "2"

    # ----------------------------------------
    #                   Test
    # ----------------------------------------
    # Initialize
    if opt.no_gpu:
        generator = utils.create_generator(opt)
    else:
        generator = utils.create_generator(opt).cuda()

    '''
    parm={}
    for name,parameters in generator.named_parameters():
        print(name,':',parameters.size())
        parm[name]=parameters.detach().cpu().numpy()
    print(parm['conv_final.weight'])
    print(parm['conv_final.bias'])
    '''

    test_dataset = dataset.DenoisingValDataset(opt)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = opt.test_batch_size, shuffle = False, num_workers = opt.num_workers, pin_memory = True)
    sample_folder = opt.save_name
    utils.check_path(sample_folder)

    psnr_sum, psnr_ave, ssim_sum, ssim_ave, time_sum, time_ave, eval_cnt = 0, 0, 0, 0, 0, 0, 0
    
    # Count start time
    prev_time = time.time()
    
    # forward
    for i, (true_input, true_target, height_origin, width_origin) in enumerate(test_loader):
        prev_time = time.time()

        # To device
        if opt.no_gpu:
            true_input = true_input
            true_target = true_target
        else:
            true_input = true_input.cuda()
            true_target = true_target.cuda()            

        # Forward propagation
        with torch.no_grad():
            #print(true_input.size()) 
            fake_target, featuremap = generator(true_input, true_input)
        
        #print(fake_target.shape, true_input.shape)
        
        time_iter = time.time() - prev_time
        time_sum = time_sum + time_iter
        #prev_time = time.time()

        # Save
        img_list = [true_input, fake_target, true_target]
        name_list = ['in', 'pred', 'gt']
        sample_name = '%d' % (i+1)
        utils.save_sample_png(sample_folder = sample_folder, sample_name = '%d' % (i + 1), img_list = img_list, name_list = name_list, pixel_max_cnt = 255, height = height_origin, width = width_origin)
        
        # Evaluation
        #psnr_sum = psnr_sum + utils.psnr(cv2.imread(sample_folder + '/' + sample_name + '_' + name_list[1] + '.png').astype(np.float32), cv2.imread(sample_folder + '/' + sample_name + '_' + name_list[2] + '.png').astype(np.float32))
        img_pred_recover = utils.recover_process(fake_target, height = height_origin, width = width_origin)
        img_gt_recover = utils.recover_process(true_target, height = height_origin, width = width_origin)
        #psnr_sum = psnr_sum + utils.psnr(utils.recover_process(fake_target, height = height_origin, width = width_origin), utils.recover_process(true_target, height = height_origin, width = width_origin))
        psnr_sum = psnr_sum + utils.psnr(img_pred_recover, img_gt_recover)
        ssim_sum = ssim_sum + compare_ssim(img_gt_recover, img_pred_recover, multichannel = True, data_range = 255)
        
        eval_cnt = eval_cnt + 1
        print('The '+ str(i) +'-th iteration: sec: ' + str(time_iter))
        
    psnr_ave = psnr_sum / eval_cnt
    ssim_ave = ssim_sum / eval_cnt
    time_ave = time_sum / eval_cnt
    fps = 1/time_ave
    eval_file = "./data/evaluation_data.txt"
    
    psnr_content = opt.load_name + ": psnr:" + str(psnr_ave) + ", ssim:" + str(ssim_ave) + ", fps: " + str(fps) + "\n"
    utils.text_save(content = psnr_content, filename = eval_file)
    
    


Namespace(add_noise=False, angle_aug=False, baseroot='./rainy_image_dataset/rain100H/test/', blind_est=True, burst_length=1, channel_att=False, color=True, core_bias=False, crop=False, crop_size=512, geometry_aug=False, init_gain=0.02, init_type='xavier', kernel_size=[3], load_name='./models/models_k3_d4_ssimloss/KPN_rainy_image_epoch250_bs16.pth', mu=0, no_gpu=False, num_workers=1, save_name='./results/models_k3_d4_ssimloss', scale_max=1, scale_min=1, sep_conv=False, sigma=30, spatial_att=False, test_batch_size=1, upMode='bilinear')
Generator is loaded!


/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


The 0-th iteration: fps: 0.0670022964477539
The 1-th iteration: fps: 0.01198577880859375
The 2-th iteration: fps: 0.0077056884765625
The 3-th iteration: fps: 0.006155967712402344
The 4-th iteration: fps: 0.011876344680786133
The 5-th iteration: fps: 0.007673740386962891
The 6-th iteration: fps: 0.007104635238647461
The 7-th iteration: fps: 0.008373022079467773
The 8-th iteration: fps: 0.007864713668823242
The 9-th iteration: fps: 0.008149385452270508
The 10-th iteration: fps: 0.0190579891204834
The 11-th iteration: fps: 0.016902685165405273
The 12-th iteration: fps: 0.008432865142822266
The 13-th iteration: fps: 0.00815272331237793
The 14-th iteration: fps: 0.00680232048034668
The 15-th iteration: fps: 0.009488821029663086
The 16-th iteration: fps: 0.008252382278442383
The 17-th iteration: fps: 0.007275819778442383
The 18-th iteration: fps: 0.0075719356536865234
The 19-th iteration: fps: 0.005666494369506836
The 20-th iteration: fps: 0.009857177734375
The 21-th iteration: fps: 0.008707